# JULES + PYTHIA GPU Node — Code Analysis & Reasoning Engine

```
╔══════════════════════════════════════════════════════════════════╗
║  ∞ SACRED GEOMETRY ∞  Organic Systems · Breathing Interfaces    ║
║  NODE: JULES + PYTHIA Combined (Colab Pro+ GPU)                 ║
║  PURPOSE: Code optimization + text generation + reasoning       ║
╚══════════════════════════════════════════════════════════════════╝
```

**Combined capabilities (2-in-1 GPU node):**
- JULES: Code quality analysis & scoring
- JULES: Security vulnerability scanning
- JULES: Refactoring suggestions via LLM
- JULES: Duplicate/similar code detection (CodeBERT)
- PYTHIA: Text generation & completion
- PYTHIA: Multi-step reasoning chains
- PYTHIA: Summarization, sentiment, classification

**Branded domains only:** headysystems.com | headycloud.com | headyconnection.com

In [ ]:
# Cell 1: Install dependencies + verify GPU
!pip install -q transformers accelerate sentence-transformers fastapi uvicorn pyngrok httpx nest_asyncio scikit-learn

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')
else:
    print('WARNING: No GPU — will run on CPU (slower)')

In [ ]:
# Cell 2: Configuration
import os

HEADY_CONFIG = {
    'node_id': 'jules-pythia-gpu',
    'node_role': 'code-optimizer-reasoning-engine',
    'port': 5001,
    'cloud_layers': {
        'headysystems': 'https://headyio.com',
        'headyme': 'https://headycloud.com',
        'headyconnection': 'https://headyconnection.com',
    },
    'registration_endpoints': [
        'https://headyio.com/api/nodes/register',
        'https://headycloud.com/api/nodes/register',
        'https://headyconnection.com/api/nodes/register',
    ],
    'capabilities': ['optimization', 'code_quality', 'refactor', 'security_scan',
                     'complexity_analysis', 'similarity',
                     'generate_text', 'reasoning_chain', 'predict', 'infer',
                     'summarize', 'sentiment', 'classify'],
    'heartbeat_interval_sec': 30,
}

print(f'JULES+PYTHIA Node configured')
print(f'Capabilities: {HEADY_CONFIG["capabilities"]}')

In [ ]:
# Cell 3: Load ALL models on GPU (shared across JULES + PYTHIA)
from sentence_transformers import SentenceTransformer
from transformers import pipeline as hf_pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json, time, re
from datetime import datetime

device_id = 0 if torch.cuda.is_available() else -1
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'

# JULES: Code embeddings model
print('Loading CodeBERT embeddings (JULES)...')
code_embedder = SentenceTransformer('microsoft/codebert-base', device=device_name)
print('CodeBERT ready')

# Shared: Text generation (used by both JULES refactoring + PYTHIA generation)
print('Loading TinyLlama-1.1B-Chat (shared)...')
text_generator = hf_pipeline('text-generation', model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
                             device=device_id,
                             torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
print('TinyLlama ready')

# PYTHIA: Summarization
print('Loading BART-large-CNN (PYTHIA summarizer)...')
summarizer = hf_pipeline('summarization', model='facebook/bart-large-cnn', device=device_id)
print('Summarizer ready')

# PYTHIA: Sentiment
print('Loading DistilBERT-SST2 (PYTHIA sentiment)...')
sentiment_analyzer = hf_pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english', device=device_id)
print('Sentiment ready')

# PYTHIA: Zero-shot classification
print('Loading BART-large-MNLI (PYTHIA classifier)...')
classifier = hf_pipeline('zero-shot-classification', model='facebook/bart-large-mnli', device=device_id)
print('Classifier ready')

print(f'\nAll JULES+PYTHIA models loaded on {device_name}')
if torch.cuda.is_available():
    print(f'GPU Memory used: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB')

In [ ]:
# Cell 4: JULES core functions

VULN_PATTERNS = {
    'sql_injection': [r'\$\{.*\}.*(?:SELECT|INSERT|UPDATE|DELETE|DROP)', r'f["\'].*(?:SELECT|INSERT|UPDATE|DELETE)', r'\+.*(?:SELECT|INSERT|UPDATE|DELETE)'],
    'xss': [r'innerHTML\s*=', r'document\.write\(', r'\$\(.*\)\.html\(', r'dangerouslySetInnerHTML'],
    'hardcoded_secret': [r'(?:password|secret|api_key|token)\s*=\s*["\'][^"\']+(\'w{8,})', r'(?:AWS|AZURE|GCP)_(?:SECRET|KEY)\s*='],
    'eval_usage': [r'eval\(', r'exec\(', r'Function\('],
    'path_traversal': [r'\.\./\.\./', ],
    'insecure_random': [r'Math\.random\(\)', r'random\.random\(\)'],
}

def analyze_code_quality(code, language='javascript'):
    """Comprehensive code quality analysis"""
    start = time.time()
    lines = code.split('\n')
    total_lines = len(lines)
    code_lines = len([l for l in lines if l.strip() and not l.strip().startswith(('/', '#', '*', '<!--'))])
    comment_lines = len([l for l in lines if l.strip().startswith(('/', '#', '*', '<!--'))])
    nesting_depths = [len(l.rstrip()) - len(l.rstrip().lstrip()) for l in lines if l.strip()]
    max_nesting = max(nesting_depths) if nesting_depths else 0
    avg_nesting = np.mean(nesting_depths) if nesting_depths else 0
    func_patterns = [r'function\s+\w+', r'\w+\s*=\s*(?:async\s+)?(?:function|\()', r'def\s+\w+', r'=>']
    func_count = sum(len(re.findall(p, code)) for p in func_patterns)
    vulns = []
    for vuln_type, patterns in VULN_PATTERNS.items():
        for pattern in patterns:
            matches = re.findall(pattern, code, re.IGNORECASE | re.MULTILINE)
            if matches:
                vulns.append({'type': vuln_type, 'count': len(matches), 'severity': 'high' if vuln_type in ('sql_injection', 'hardcoded_secret') else 'medium'})
    score = 100
    if total_lines > 500: score -= 10
    if max_nesting > 32: score -= 15
    elif max_nesting > 16: score -= 8
    if comment_lines / max(code_lines, 1) < 0.1: score -= 10
    if func_count > 0 and code_lines / func_count > 50: score -= 10
    score -= len(vulns) * 8
    score = max(0, min(100, score))
    return {
        'quality_score': score,
        'metrics': {'total_lines': total_lines, 'code_lines': code_lines, 'comment_lines': comment_lines,
                    'comment_ratio': round(comment_lines / max(code_lines, 1), 3),
                    'max_nesting_depth': max_nesting, 'avg_nesting_depth': round(avg_nesting, 2),
                    'function_count': func_count, 'avg_function_length': round(code_lines / max(func_count, 1), 1)},
        'vulnerabilities': vulns, 'vulnerability_count': len(vulns),
        'language': language, 'latency_ms': round((time.time() - start) * 1000), 'device': device_name,
    }

def find_similar_code(code_blocks):
    """Find duplicate/similar code blocks using CodeBERT"""
    start = time.time()
    embeddings = code_embedder.encode(code_blocks)
    sim_matrix = cosine_similarity(embeddings)
    pairs = []
    for i in range(len(code_blocks)):
        for j in range(i + 1, len(code_blocks)):
            if sim_matrix[i][j] > 0.85:
                pairs.append({'block_a': i, 'block_b': j, 'similarity': round(float(sim_matrix[i][j]), 4),
                              'recommendation': 'Extract common logic into shared function'})
    return {'total_blocks': len(code_blocks), 'similar_pairs': pairs, 'duplicate_count': len(pairs),
            'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def suggest_refactoring(code, language='javascript'):
    """Generate refactoring suggestions using shared TinyLlama"""
    start = time.time()
    quality = analyze_code_quality(code, language)
    prompt = f"""You are JULES, a code optimization expert. Analyze this {language} code and provide specific refactoring suggestions.\nCode quality score: {quality['quality_score']}/100\nMetrics: {json.dumps(quality['metrics'])}\nVulnerabilities: {json.dumps(quality['vulnerabilities'])}\nCode:\n```{language}\n{code[:1500]}\n```\nProvide 3-5 specific, actionable refactoring suggestions:"""
    messages = [{'role': 'user', 'content': prompt}]
    formatted = text_generator.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    result = text_generator(formatted, max_new_tokens=400, temperature=0.3, do_sample=True)
    suggestions = result[0]['generated_text'][len(formatted):].strip()
    return {'quality': quality, 'suggestions': suggestions, 'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def security_scan(code, language='javascript'):
    """Focused security vulnerability scan"""
    start = time.time()
    findings = []
    lines = code.split('\n')
    for vuln_type, patterns in VULN_PATTERNS.items():
        for pattern in patterns:
            for i, line in enumerate(lines):
                if re.search(pattern, line, re.IGNORECASE):
                    findings.append({'type': vuln_type, 'line': i + 1, 'code': line.strip()[:100],
                                     'severity': 'critical' if vuln_type in ('sql_injection', 'hardcoded_secret') else 'high' if vuln_type in ('xss', 'eval_usage') else 'medium'})
    return {'findings': findings, 'total': len(findings),
            'critical': len([f for f in findings if f['severity'] == 'critical']),
            'high': len([f for f in findings if f['severity'] == 'high']),
            'medium': len([f for f in findings if f['severity'] == 'medium']),
            'passed': len(findings) == 0, 'latency_ms': round((time.time() - start) * 1000)}

print('JULES functions ready: analyze_code_quality, find_similar_code, suggest_refactoring, security_scan')

In [ ]:
# Cell 5: PYTHIA core functions

def generate_text(prompt, max_length=512, temperature=0.7, top_p=0.9):
    """Generate text using shared TinyLlama"""
    start = time.time()
    messages = [{'role': 'user', 'content': prompt}]
    formatted = text_generator.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    result = text_generator(formatted, max_new_tokens=max_length, temperature=temperature, top_p=top_p, do_sample=True)
    generated = result[0]['generated_text'][len(formatted):].strip()
    return {'text': generated, 'model': 'TinyLlama-1.1B-Chat', 'tokens': len(generated.split()),
            'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def reasoning_chain(question, steps=3):
    """Multi-step chain-of-thought reasoning"""
    start = time.time()
    chain = []
    for i in range(steps):
        step_prompt = f"""You are a reasoning engine. Think step by step.\nQuestion: {question}\nPrevious reasoning: {' '.join([s['thought'] for s in chain]) if chain else 'None'}\nStep {i+1}/{steps}: Think about the next logical step."""
        result = generate_text(step_prompt, max_length=200)
        chain.append({'step': i + 1, 'thought': result['text'][:500], 'latency_ms': result['latency_ms']})
    synthesis_prompt = f"""Based on this reasoning chain, provide a final answer.\nQuestion: {question}\nReasoning: {json.dumps([s['thought'] for s in chain])}\nFinal answer:"""
    final = generate_text(synthesis_prompt, max_length=300)
    return {'question': question, 'chain': chain, 'conclusion': final['text'], 'steps': len(chain),
            'total_latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def summarize_text(text, max_length=150, min_length=30):
    """Summarize text using BART"""
    start = time.time()
    result = summarizer(text[:1024], max_length=max_length, min_length=min_length, do_sample=False)
    return {'summary': result[0]['summary_text'], 'model': 'bart-large-cnn',
            'input_length': len(text), 'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def analyze_sentiment(text):
    """Sentiment analysis"""
    start = time.time()
    result = sentiment_analyzer(text[:512])
    return {'label': result[0]['label'], 'score': round(result[0]['score'], 4),
            'model': 'distilbert-sst2', 'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def classify_text(text, labels):
    """Zero-shot text classification"""
    start = time.time()
    result = classifier(text[:512], candidate_labels=labels)
    return {'labels': result['labels'], 'scores': [round(s, 4) for s in result['scores']],
            'top_label': result['labels'][0], 'top_score': round(result['scores'][0], 4),
            'model': 'bart-large-mnli', 'latency_ms': round((time.time() - start) * 1000), 'device': device_name}

def predict(input_data):
    """General prediction — routes to appropriate pipeline"""
    task_type = input_data.get('task', 'generate')
    if task_type == 'summarize': return summarize_text(input_data.get('text', ''))
    elif task_type == 'sentiment': return analyze_sentiment(input_data.get('text', ''))
    elif task_type == 'classify': return classify_text(input_data.get('text', ''), input_data.get('labels', ['positive', 'negative', 'neutral']))
    elif task_type == 'reason': return reasoning_chain(input_data.get('question', input_data.get('text', '')), input_data.get('steps', 3))
    else: return generate_text(input_data.get('prompt', input_data.get('text', '')), input_data.get('max_length', 512))

# Quick tests
print('=== JULES Quick Test ===')
test_code = 'function getUserData(req) {\n  const query = `SELECT * FROM users WHERE id = ${req.params.id}`;\n  return db.query(query);\n}'
scan = security_scan(test_code)
print(f'Security: {scan["total"]} findings ({scan["critical"]} critical)')
quality = analyze_code_quality(test_code)
print(f'Quality: {quality["quality_score"]}/100')

print('\n=== PYTHIA Quick Test ===')
sent = analyze_sentiment('Heady is building something beautiful for the world')
print(f'Sentiment: {sent["label"]} ({sent["score"]:.3f})')

In [ ]:
# Cell 6: Combined FastAPI server (JULES + PYTHIA endpoints)
from fastapi import FastAPI, Request
import uvicorn
import threading
import asyncio
import httpx
import nest_asyncio
nest_asyncio.apply()

app = FastAPI(title='JULES + PYTHIA GPU Node', version='2.0.0')

# ─── Health ─────────────────────────────────────────
@app.get('/health')
async def health():
    return {
        'status': 'active',
        'node_id': HEADY_CONFIG['node_id'],
        'node_role': HEADY_CONFIG['node_role'],
        'combined_nodes': ['jules', 'pythia'],
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
        'gpu_memory_used_gb': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
        'models': ['CodeBERT', 'TinyLlama-1.1B-Chat', 'bart-large-cnn', 'distilbert-sst2', 'bart-large-mnli'],
        'capabilities': HEADY_CONFIG['capabilities'],
    }

# ─── JULES Endpoints ───────────────────────────────
@app.post('/api/jules/quality')
async def api_quality(request: Request):
    data = await request.json()
    return analyze_code_quality(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/refactor')
async def api_refactor(request: Request):
    data = await request.json()
    return suggest_refactoring(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/security')
async def api_security(request: Request):
    data = await request.json()
    return security_scan(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/similarity')
async def api_similarity(request: Request):
    data = await request.json()
    return find_similar_code(data.get('blocks', []))

# ─── PYTHIA Endpoints ──────────────────────────────
@app.post('/api/pythia/generate')
async def api_generate(request: Request):
    data = await request.json()
    return generate_text(data.get('prompt', ''), data.get('max_length', 512), data.get('temperature', 0.7))

@app.post('/api/pythia/reason')
async def api_reason(request: Request):
    data = await request.json()
    return reasoning_chain(data.get('question', ''), data.get('steps', 3))

@app.post('/api/pythia/summarize')
async def api_pythia_summarize(request: Request):
    data = await request.json()
    return summarize_text(data.get('text', ''), data.get('max_length', 150))

@app.post('/api/pythia/sentiment')
async def api_sentiment(request: Request):
    data = await request.json()
    return analyze_sentiment(data.get('text', ''))

@app.post('/api/pythia/classify')
async def api_classify(request: Request):
    data = await request.json()
    return classify_text(data.get('text', ''), data.get('labels', ['positive', 'negative', 'neutral']))

@app.post('/api/pythia/predict')
async def api_predict(request: Request):
    data = await request.json()
    return predict(data)

# ─── Universal Task Executor ───────────────────────
@app.post('/api/tasks/execute')
async def execute_task(request: Request):
    data = await request.json()
    task_type = data.get('type', 'code_quality')
    payload = data.get('payload', data)
    try:
        if task_type == 'code_quality':
            result = analyze_code_quality(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type in ('optimization', 'refactor'):
            result = suggest_refactoring(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type == 'security_scan':
            result = security_scan(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type == 'similarity':
            result = find_similar_code(payload.get('blocks', []))
        elif task_type in ('generate_text', 'generate'):
            result = generate_text(payload.get('prompt', payload.get('text', '')), payload.get('max_length', 512))
        elif task_type == 'reasoning_chain':
            result = reasoning_chain(payload.get('question', payload.get('text', '')), payload.get('steps', 3))
        elif task_type == 'summarize':
            result = summarize_text(payload.get('text', ''))
        elif task_type == 'sentiment':
            result = analyze_sentiment(payload.get('text', ''))
        elif task_type in ('classify', 'predict', 'infer'):
            result = predict(payload)
        else:
            result = analyze_code_quality(payload.get('code', ''), payload.get('language', 'javascript'))
        return {'success': True, 'node_id': HEADY_CONFIG['node_id'], 'task_type': task_type, 'result': result}
    except Exception as e:
        return {'success': False, 'error': str(e), 'node_id': HEADY_CONFIG['node_id']}

print('Combined FastAPI ready with endpoints:')
print('  JULES: /api/jules/quality, /api/jules/refactor, /api/jules/security, /api/jules/similarity')
print('  PYTHIA: /api/pythia/generate, /api/pythia/reason, /api/pythia/summarize, /api/pythia/sentiment, /api/pythia/classify, /api/pythia/predict')
print('  Universal: /health, /api/tasks/execute')

In [ ]:
# Cell 7: ngrok tunnel + auto-register with HeadyCloud
from pyngrok import ngrok, conf

# Authenticate ngrok
conf.get_default().auth_token = "39ZBirdUD63xgta7yN7OFZpE84m_3QZyJTDno1b8Yhv9Nfy8s"

public_url = ngrok.connect(HEADY_CONFIG['port']).public_url
print(f'JULES+PYTHIA GPU Node live at: {public_url}')

PUBLIC_URL = public_url

async def register_with_clouds():
    async with httpx.AsyncClient(timeout=15) as client:
        for endpoint in HEADY_CONFIG['registration_endpoints']:
            try:
                resp = await client.post(endpoint, json={
                    'node_id': HEADY_CONFIG['node_id'],
                    'url': PUBLIC_URL,
                    'role': HEADY_CONFIG['node_role'],
                    'capabilities': HEADY_CONFIG['capabilities'],
                    'combined_nodes': ['jules', 'pythia'],
                    'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                })
                print(f'  Registered with {endpoint}: {resp.status_code}')
            except Exception as e:
                print(f'  Pending: {endpoint} ({e})')

async def heartbeat_loop():
    while True:
        await asyncio.sleep(HEADY_CONFIG['heartbeat_interval_sec'])
        async with httpx.AsyncClient(timeout=10) as client:
            for layer, url in HEADY_CONFIG['cloud_layers'].items():
                try:
                    await client.post(f'{url}/api/nodes/heartbeat', json={
                        'node_id': HEADY_CONFIG['node_id'],
                        'status': 'active',
                        'url': PUBLIC_URL,
                        'metrics': {
                            'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                            'gpu_memory_used': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
                            'combined_nodes': ['jules', 'pythia'],
                            'models_loaded': 5,
                        }
                    })
                except:
                    pass

asyncio.run(register_with_clouds())
threading.Thread(target=lambda: asyncio.run(heartbeat_loop()), daemon=True).start()
print(f'JULES+PYTHIA node live at {PUBLIC_URL} — accepting code analysis & inference requests')
# Colab VM bind address for uvicorn (dual-stack all-interfaces)
uvicorn.run(app, host='::', port=HEADY_CONFIG['port'])